In [1]:
# default_exp inference

In [2]:
#all_slow

In [3]:
#hide
%reload_ext autoreload
%autoreload 2
%matplotlib inline

# inference

> All inference related methods

In [4]:
#export
import datetime, warnings, gc
from inspect import signature
import sklearn.metrics as skm

from fastai.text.all import *

from tritonlytics_ai.utils import *
from tritonlytics_ai.verbatims.core import *

In [5]:
#hide
import pdb

from nbdev.showdoc import *
from fastcore.test import *

In [6]:
#hide
from fastai import __version__ as fa_version
from torch import __version__ as pt_version
from transformers import __version__ as hft_version

print(f'Using pytorch {pt_version}')
print(f'Using fastai {fa_version}')
print(f'Using transformers {hft_version}')

Using pytorch 1.6.0
Using fastai 2.0.16
Using transformers 3.3.1


In [7]:
#cuda
torch.cuda.set_device(1)
print(f'Using GPU #{torch.cuda.current_device()}: {torch.cuda.get_device_name()}')

Using GPU #1: GeForce GTX 1080 Ti


## Utility 

In [8]:
#export
def concat_pool(raw_outputs):
    last_rnn_layer = raw_outputs[:,-1,None] # (e.g. (bs,n_hid,emb_sz) => (bs,1,emb_sz)) 
    bsz = last_rnn_layer.shape[0] 
    
    avg_pool = F.adaptive_avg_pool1d(last_rnn_layer.permute(0,2,1), 1).view(bsz, -1)
    max_pool = F.adaptive_max_pool1d(last_rnn_layer.permute(0,2,1), 1).view(bsz, -1)
    last_outp = last_rnn_layer[:,-1,:]

    return torch.cat([last_outp, max_pool, avg_pool], 1)

## What models should be run?

In [9]:
f = json.loads((RAW_DATA_PATH/'verbatim-inference.json').read_text())

In [10]:
models_df = pd.DataFrame(f['models'])
models_df.head()

,model_run_id,survey_id,model_id,model_name,model_type_id,model_type_name
0,124,396,5,20200511-ulmfit-verbatim-standard-themes-saw,4,verbatim-classification-saw-themes
1,125,396,4,20200511-ulmfit-verbatim-sent,2,verbatim-classification-sentiment


In [11]:
inf_df = pd.DataFrame(f['data'])
print(len(inf_df));

698


In [12]:
yyyymmdd = datetime.today().strftime("%Y%m%d")

device = torch.device('cuda:1') if torch.cuda.is_available() else torch.device('cpu')
print(device)

cuda:1


## Sentiment

In [13]:
from tritonlytics_ai.verbatims.sentiment.training import get_sentiment_preds

sent_df, inf_probs, inf_labels = get_sentiment_preds(inf_df, device=device)
print(sent_df.shape, inf_probs.shape, len(inf_labels))

0
(695, 90) torch.Size([695, 8]) 8


## Standard Themes - S@W

In [14]:
from tritonlytics_ai.verbatims.standard_themes_saw.training import get_standard_theme_preds as get_saw_theme_preds

saw_themes_df, inf_probs, inf_labels = get_saw_theme_preds(inf_df, device=device)
print(saw_themes_df.shape, inf_probs.shape, len(inf_labels))

0
(695, 124) torch.Size([695, 25]) 25


In [15]:
pred_cols = [f'pred_{lbl}' for lbl in inf_labels]
saw_themes_df[['answer_text']+pred_cols].head(2)

,answer_text,pred_adequate_staffing,pred_advancement_and_training_opportunities,pred_appropriate_stress_work_assigned_equitably,pred_benefits,pred_better_ways_recognized_participate_in_decisions,pred_career_advancement,pred_committed_to_diversity,pred_communicates_essential_information,pred_ethical_conduct_perform_responsibilities_spirit_of_cooperation,...,pred_feel_valued_by_department,pred_flexibility_work_life_balance,pred_good_use_of_skills,pred_have_necessary_tools,pred_have_voice_within_my_institution_valued_member_of_my_institution,pred_internal_processes_effective,pred_parking_transportation,pred_salary_pay,pred_satisfied_with_diversity_progams,pred_supervisor_effectiveness_resolves_staff_issues
0,"I believe ANR leadership is completely disconnected from the work of campus-based specialists. When I am responding regarding ""my department"" in this survey it is my campus department I am evaluating, not an ANR department.",0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,I'm a new employee.,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Standard Themes - CSS

In [16]:
from tritonlytics_ai.verbatims.standard_themes_css.training import get_standard_theme_preds as get_css_theme_preds

css_themes_df, inf_probs, inf_labels = get_css_theme_preds(inf_df, device=device)
print(css_themes_df.shape, inf_probs.shape, len(inf_labels))

0
(695, 106) torch.Size([695, 16]) 16


In [17]:
pred_cols = [f'pred_{lbl}' for lbl in inf_labels]
css_themes_df[['answer_text']+pred_cols].head(2)

,answer_text,pred_accessible_to_customers,pred_consistency_in_policies_information,pred_cost_fees,pred_courteous_professional_staff,pred_effective_communications,pred_effectively_uses_websites_online_documentation,pred_helpful_staff,pred_knowledgeable_staff,pred_moving_in_a_positive_direction,pred_overall_satisfaction,pred_process_improvement,pred_provides_effective_advice_guidance,pred_provides_training_on_processes_applications,pred_resolves_problems_effectively,pred_responds_to_requests_within_an_acceptable_time,pred_understands_my_needs_and_requirements
0,"I believe ANR leadership is completely disconnected from the work of campus-based specialists. When I am responding regarding ""my department"" in this survey it is my campus department I am evaluating, not an ANR department.",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,I'm a new employee.,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## Standard Themes - Meta

In [18]:
from tritonlytics_ai.verbatims.standard_themes_meta.training import get_standard_theme_meta_preds, build_meta_inf_df
from tritonlytics_ai.verbatims.standard_themes_meta.training import sentiment_mse, is_example_acc
from tritonlytics_ai.verbatims.standard_themes_meta.training import Meta_MM, Meta_MM_HF_BaseModelCallback

meta_inf_df = build_meta_inf_df(saw_themes_df, fixed_cols=list(TASK_LM_DTYPES_SC.keys()))
meta_df, inf_probs_sent, inf_probs_is_example, inf_labels = get_standard_theme_meta_preds(meta_inf_df, device=device)
print(meta_df.shape, inf_probs_sent.shape, inf_probs_is_example.shape, len(inf_labels))

0
(843, 55) torch.Size([843, 1]) torch.Size([843, 2]) 2


In [19]:
meta_df[['theme', 'answer_text', 'prob_avg_sentiment', 'pred_is_example']].head()

,theme,answer_text,prob_avg_sentiment,pred_is_example
0,Adequate Staffing,"While our department has a good work environment as a whole there are several area's that are a major issues.\r\n1) We have zero leadership, we have no direct managers over us. While we are able to manage our own work, we have very little real direction or support and it directly ties into other issues here.\r\n2) There is a Lone Ranger in our department. This employee has control over the servers without oversight and without transparency. This person has taken advantage of their role and privileged access to assert authority over others when this person has none. There is no collaboratio...",2.494204,0
1,Adequate Staffing,"In my department, I and nearly all of us are 100% on ""soft"" money. Further, we have very little support staff. The lack of job security is highly problematic both for planning ahead for work strategies as well as for personal stress level. The lack of support hinders my ability to do my best and to provide the best quality output.",2.513639,0
2,Adequate Staffing,There is an overall situation of understaffed and overwhelmed. Having to oversee multiple counties with no staff/help in one county is detrimental.,2.317858,0
3,Adequate Staffing,The biggest issue I see within UC ANR is inequitable pay and insufficient staffing that often results in inequitable distribution of work throughout my unit.,2.087559,0
4,Adequate Staffing,"I am increasingly discouraged by leadership's unwillingness to address the problem of asking academics to work outside their position description and training. Specifically, I find it incredibly frustrating to be required to cover 4-H programs to the detriment of my own clientele. So frustrated, in fact, that I've found myself considering other job opportunities, despite the fact that when I was hired, I'd hoped I'd retire in this job.\r\n\r\nLeadership also has created an ""us"" versus ""you all"" environment. I have not observed any of our leadership taking responsibility for problems or cha...",2.318836,0


## Cleanup

In [20]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_utils.ipynb.
Converted 02a_verbatims-core.ipynb.
Converted 02b_verbatims-sentiment.ipynb.
Converted 02c_verbatims-standard-themes-saw-training.ipynb.
Converted 02d_verbatims-standard-themes-css-training.ipynb.
Converted 02e_verbatims-standard-themes-meta-training.ipynb.
Converted 99_verbatims-inference.ipynb.
Converted index.ipynb.
